# Custom Layers

## Layers without Parameters

In [1]:
from mxnet import gluon, nd
from mxnet.gluon import nn

class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)

    def forward(self, x):
        return x - x.mean()

### Forward

In [2]:
layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

### Construct More Complex Models

In [3]:
net = nn.Sequential()
net.add(nn.Dense(128), CenteredLayer())
net.initialize()

y = net(nd.random.uniform(shape=(4, 8)))
y.mean().asscalar()

-6.0936145e-10

## Layers with Parameters

In [4]:
params = gluon.ParameterDict()
params.get('param2', shape=(2, 3))
params

(
  Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

### A Customized Dense Layer

In [5]:
class MyDense(nn.Block):
    # Units: the number of outputs in this layer; in_units: the number of inputs in this layer.
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units,))

    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

### Forward

In [6]:
dense = MyDense(units=3, in_units=5)
print(dense.params)
dense.initialize()
dense(nd.random.uniform(shape=(2, 5)))

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)



[[0.06917784 0.01627153 0.01029644]
 [0.02602214 0.04537309 0.        ]]
<NDArray 2x3 @cpu(0)>

### Use it to Construct Models

In [7]:
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
        MyDense(1, in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.03820475]
 [0.04035058]]
<NDArray 2x1 @cpu(0)>